 # Building a full hourly load profile from daily and monthly data

This notebook builds a full 8760-hour load profile from daily hourly shape data and monthly scaling coefficients. The output is a long-format DataFrame with columns: season, day, hour, zone, value.

It can be used when you have daily load profiles (e.g., hourly shape for each day) and monthly scaling coefficients (e.g., how much the load changes each month).
Assuming the daily data is in a CSV file with hourly values for each day, and the monthly data is in another CSV file with scaling coefficients for each month.

In [2]:
import os
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
daily = 'input/load_daily.csv'
daily = pd.read_csv(daily, encoding='utf-8', low_memory=False, index_col=[0], header=[0])
monthly = 'input/load_monthly.csv'
monthly = pd.read_csv(monthly, encoding='utf-8', low_memory=False, index_col=[0], header=[0])

In [6]:
def build_hourly_profile(hourly_df, monthly_df):
    """
    Build a full 8760-hour profile from daily hourly shape and monthly scaling coefficients.
    Returns a long-format DataFrame with columns: season, day, hour, zone, value.
    """
    # Ensure index is sorted and labeled properly
    hourly_df = hourly_df.copy()
    monthly_df = monthly_df.copy()


    # Day-of-year and month mapping
    date_range = pd.date_range("2025-01-01", "2026-01-01", freq="h", inclusive="left")
    calendar = pd.DataFrame({
        "datetime": date_range,
        "day": date_range.day,
        "month": date_range.month,
        "hour": date_range.hour + 1  # To match 1–24 indexing
    })


    # Multiply hourly profile by monthly coefficient for each zone
    result = []

    for zone in hourly_df.columns:
        # Check if the zone exists in monthly_df
        if zone not in monthly_df.columns:
            raise ValueError(f"Zone '{zone}' not found in monthly coefficients.")

        # Build a daily profile for the full year for this zone
        daily_values = hourly_df[zone].reindex(calendar["hour"].values).reset_index(drop=True)
        month_coeffs = calendar["month"].map(monthly_df[zone]).reset_index(drop=True)
        full_year_profile = daily_values * month_coeffs

        zone_df = calendar.copy()
        zone_df["value"] = full_year_profile
        zone_df["zone"] = zone
        result.append(zone_df)

    final_df = pd.concat(result, ignore_index=True)
    # Normalize the 'value' column to ensure the max is 1 for each zone
    final_df["value"] = final_df.groupby("zone")["value"].transform(lambda x: x / x.max())


    return final_df[["zone", "month", "day", "hour", "value"]]

load = build_hourly_profile(daily, monthly)
load.to_csv('output/load_full_year.csv', index=False)
display(load.head(), load.tail())

,zone,month,day,hour,value
0,Angola,1,1,1,0.797619
1,Angola,1,1,2,0.767429
2,Angola,1,1,3,0.756716
3,Angola,1,1,4,0.735290
4,Angola,1,1,5,0.728473


,zone,month,day,hour,value
96355,Chad,12,31,20,0.809524
96356,Chad,12,31,21,0.781190
96357,Chad,12,31,22,0.729381
96358,Chad,12,31,23,0.668667
96359,Chad,12,31,24,0.607952


In [7]:
def plot_generation_profiles(result_df, target_day=15, output_prefix="generation"):

    # Ensure datetime is available
    if "datetime" not in result_df.columns:
        result_df = result_df.copy()
        result_df["datetime"] = pd.to_datetime("2025-01-01") + pd.to_timedelta(
            (result_df["day"] - 1) * 24 + (result_df["hour"] - 1), unit="h"
        )

    zones = result_df["zone"].unique()

    # 1. Plot for a specific day (subplots by country)
    fig, axes = plt.subplots(len(zones), 1, figsize=(10, 3 * len(zones)), sharex=True)
    fig.suptitle(f"Hourly Profile – Day {target_day}", fontsize=14)
    for i, zone in enumerate(zones):
        ax = axes[i] if len(zones) > 1 else axes
        day_df = result_df[(result_df["day"] == target_day) & (result_df["zone"] == zone)]
        ax.plot(day_df["hour"], day_df["value"], marker="o")
        ax.set_title(zone)
        ax.set_ylabel("Value")
        ax.set_xlim(1, 24)
        if i == len(zones) - 1:
            ax.set_xlabel("Hour")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f"output/{output_prefix}_day{target_day}.png")
    plt.close()

    # 2. Monthly average profile (one plot, all countries)
    monthly_avg = (
        result_df.groupby(["month", "hour", "zone"])["value"]
        .mean()
        .reset_index()
    )
    plt.figure(figsize=(10, 6))
    for zone in zones:
        zdf = monthly_avg[monthly_avg["zone"] == zone]
        avg_by_hour = zdf.groupby("hour")["value"].mean()
        plt.plot(avg_by_hour.index, avg_by_hour.values, label=zone)
    plt.title("Average Daily Profile Over the Year (All Countries)")
    plt.xlabel("Hour")
    plt.ylabel("Load")
    # Legend on the right side
    plt.legend(title="Zone", bbox_to_anchor=(1.05, 1), loc='upper left')
    # Only horizontal grid lines
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f"output/{output_prefix}_monthly_average.png")
    plt.close()

    # 3.



    print("Plots saved:")
    print(f"- {output_prefix}_day{target_day}.png")
    print(f"- {output_prefix}_monthly_average.png")
    print(f"- {output_prefix}_yearly_by_country.png")

plot_generation_profiles(load, target_day=15, output_prefix="load")

Plots saved:
- load_day15.png
- load_monthly_average.png
- load_yearly_by_country.png
